In [ ]:
# Import packages
import pyspark
import dxpy
import dxdata


In [ ]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)


In [ ]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']


In [ ]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)


In [ ]:
participant = dataset['participant']


In [ ]:
field_name_dict = {
    'sample_names': 'eid',
    'sex': 'p31',
    'genetic_sex': 'p22001',
    'sex_chromosome_aneuploidy': 'p22019',
    'genetic_kinship_to_other_participants': 'p22021',
    'out_hetz_missing': 'p22027',
}
    

In [ ]:
field_names = list(field_name_dict.values())


In [ ]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="replace")


In [ ]:
def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    return


In [ ]:
pandas_df = df.toPandas()
pandas_df = pandas_df.rename(columns={v:k for k,v in field_name_dict.items()})



In [ ]:
pandas_df.head()


In [ ]:
proj_dir = f"/fields/data/sample_qc/"
filename = "sample_qc_info.tsv"
pandas_df.to_csv(filename, index=False, sep="\t")
upload_file_to_project(filename, proj_dir)
